# Extensions

The purpose of this notebook is to test everything related to merging initial hits

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from testing_framework import testing_utils
import database
from preprocessing import preprocessing_utils, merge_search, clustering
from identification import create_hits
from gen_spectra import get_precursor, gen_spectrum
import utils

import operator

max_peptide_length = 23
ppm_tolerance = 20
precursor_tolerance = 10
peak_filter = 25
relative_abundance_filter = 0.1

import matplotlib.pyplot as plt

In [2]:
datasets = testing_utils.define_data()

dataset = datasets[0]

input_spectra_path = [os.path.join(dataset[0], 'NOD2_E3.mzML')]
input_spectra, boundaries = preprocessing_utils.load_spectra(input_spectra_path, ppm_tolerance, peak_filter=peak_filter, relative_abundance_filter=relative_abundance_filter)

correct_sequences = testing_utils.generate_truth_set(datasets[0])

path = dataset[2]
db = database.build(path)

In [3]:
write_path = os.path.abspath(os.path.join(module_path, 'intermediate_files'))
matched_masses_b, matched_masses_y, kmer_set = merge_search.modified_match_masses(boundaries, db, max_peptide_length, True, write_path)
print('Finished matching masses')

On protein 279/279 [100%]
Sorting the set of protein masses...
Sorting the set of protein masses done
Performing Merge
Done
Finished matching masses


In [4]:
print(len(matched_masses_b), len(matched_masses_y))

20410 20364


In [5]:
unique_b,unique_y = testing_utils.get_unique_matched_masses(boundaries, matched_masses_b, matched_masses_y)

# Getting initial hits

In [6]:
for i, seq in enumerate(correct_sequences):
    if len(seq) > 30:
        print(i)

92
97
98
99
100
148
153
154
199
202
206
248
249
251
263
266
269
277
278
309
318
323
445
449
453
458
460
483
498
518
519
528
596
597
600
603
604
605
606
679
689
695
699
703
704
705
706
707
708
709
711
712
713
714
715
716
745
768
784
802
854
855
898
905
906
950
953
954
1023
1032
1040
1041
1042
1074
1079


In [7]:
spectrum_num = 6

correct_sequence = correct_sequences[spectrum_num]
print(correct_sequence)

input_spectrum = input_spectra[spectrum_num]

DLPVNSPMTKG


In [8]:
location = os.path.join(os.path.abspath(os.path.join('../..')), 'intermediate_files/')
b_hits, y_hits = create_hits(spectrum_num, input_spectrum, matched_masses_b, matched_masses_y, False, location)
correct_hits = testing_utils.append_correct_hits(correct_sequence, input_spectrum, ppm_tolerance)
ion = 'b'
clusters = testing_utils.create_clusters(ion, b_hits, y_hits)
b_sorted_clusters = testing_utils.Bayes_clusters(ion, clusters, location, kmer_set, unique_b)
ion = 'y'
clusters = testing_utils.create_clusters(ion, b_hits, y_hits)
y_sorted_clusters = testing_utils.Bayes_clusters(ion, clusters, location, kmer_set, unique_y)
with open(os.path.join(location, 'b_sorted_clusters.txt'), 'w') as b:
    [b.write(str(x) + '\n') for x in b_sorted_clusters]
with open(os.path.join(location, 'y_sorted_clusters.txt'), 'w') as y:
    [y.write(str(x) + '\n') for x in y_sorted_clusters]
print(len(b_hits), len(y_hits), len(boundaries), len(input_spectra))
print(len(clusters), len(b_sorted_clusters), len(y_sorted_clusters))

ValueError: invalid literal for int() with base 10: ''

# Printing hits

In [ ]:
b_sorted_clusters = sorted(b_sorted_clusters, key=operator.attrgetter('score', 'pid'), reverse = True)
testing_utils.write_b_sorted_cluster(b_sorted_clusters)
for i in range(0, 50):
    x = b_sorted_clusters[i]
    score = x.score
    seq = x.seq
    print(score, seq)

In [ ]:
y_sorted_clusters = sorted(y_sorted_clusters, key=operator.attrgetter('prob', 'score', 'pid'))
# y_sorted_clusters = sorted(y_sorted_clusters, key=operator.attrgetter('score', 'prob', 'pid'), reverse = True)
testing_utils.write_y_sorted_cluster(y_sorted_clusters)
for i in range(0, 50):
    x = y_sorted_clusters[i]
    score = x.score
    seq = x.seq
    print(score, seq)

# Merging

In [ ]:
m = clustering.Ryan_merge(b_sorted_clusters, y_sorted_clusters)
m.sort(key = lambda x: x[0], reverse = True)
[print(m[x]) for x in range(0,10)]
print(len(m))

In [ ]:
def filter_by_precursor(mseqs, obs_prec, precursor_tol, charge):
    filtered_seqs = []
    for comb_seq in mseqs:
        b_seq = comb_seq[3][4]
        y_seq = comb_seq[4][4]
        if b_seq != y_seq:
            new_seq = b_seq + y_seq
        else:
            new_seq = b_seq
        tol = utils.ppm_to_da(obs_prec, precursor_tol)
        if not (get_precursor(new_seq, charge) > obs_prec + tol):
            filtered_seqs.append(comb_seq)
    return filtered_seqs

m = filter_by_precursor(m, input_spectrum.precursor_mass, precursor_tolerance, input_spectrum.precursor_charge)
[print(m[x]) for x in range(0,10)]

In [ ]:
print(len(m))
print(m[:50])

# Hybrid Merge

In [ ]:
from constants import WATER_MASS, PROTON_MASS
import collections

def min_info(cluster):
    return (cluster.pid, cluster.start, cluster.end, cluster.score, cluster.seq)

def check_for_hybrid_overlap(b_seq, y_seq, ion):
    match = True
    print('code got here')
    if ion == 'b':
        for i, char in enumerate(b_seq):
            if char == y_seq[0]:
                k = 0
                for j in range(i, len(b_seq) + 1):
                    if b_seq[j] != y_seq[k]:
                        match = False
                        break
        if match == True:
            print('Match was true for', b_seq)
            modified_seq = b_seq[:i]
    else:
        for i, char in enumerate(y_seq):
            if char == y_seq[0]:
                k = 0
                for j in range(i, len(b_seq) + 1):
                    if b_seq[j] != y_seq[k]:
                        match = False
                        break
        if match == True:
            print('Match was true for', b_seq)
            modified_seq = b_seq[:i]
    return match, modified_seq

def grab_matches(b,indexed_clusters, target_val, ion):
    #Given a cluster we want to find everything that it can pair with
    # It can pair with anything up to a certain mass 
    current_index = 0
    matches = []
    for key in indexed_clusters.keys():
        if key<=target_val: #if key is a valid key
            for y in indexed_clusters[key]:
                if ion == 'b':
                    matches.append((b.score + y.score, b.end - y.start, y.end-b.start,min_info(b), min_info(y)))
                else:
                    matches.append((b.score + y.score, b.end - y.start, y.end-b.start,min_info(y), min_info(b)))
        else:
#             match, modified_seq = check_for_hybrid_overlap()
            break            
    return matches
    
def index_by_precursor_mass(sorted_clusters, pc):
    indexed = dict()
    for y in sorted_clusters:
        if get_precursor(y.seq, pc) not in indexed.keys():
            indexed[get_precursor(y.seq, pc)] = []
        indexed[get_precursor(y.seq, pc)].append(y)
    indexed = collections.OrderedDict(sorted(indexed.items(),key=lambda t: t[0]))
    return indexed
    
def get_hybrid_matches(b_sorted_clusters, y_sorted_clusters, obs_prec, precursor_tol, charge):
    merged_seqs = []
    ind_b, ind_y = index_by_precursor_mass(b_sorted_clusters, charge),index_by_precursor_mass(y_sorted_clusters, charge)
    for i, cluster in enumerate(b_sorted_clusters[:10]):
        cluster_seq = cluster.seq
        cluster_mass = get_precursor(cluster_seq, charge)
        tol = utils.ppm_to_da(obs_prec, precursor_tol)
        if not (cluster_mass > obs_prec + tol):
            diff = obs_prec + tol - cluster_mass + (charge * PROTON_MASS) + WATER_MASS
            merges = grab_matches(cluster,ind_y, diff, 'b')
            [merged_seqs.append(x) for x in merges]
    for i, cluster in enumerate(y_sorted_clusters[:10]):
        cluster_seq = cluster.seq
        cluster_mass = get_precursor(cluster_seq, charge)
        tol = utils.ppm_to_da(obs_prec, precursor_tol)
        if not (cluster_mass > obs_prec + tol):
            diff = obs_prec + tol - cluster_mass + (charge * PROTON_MASS) + WATER_MASS
#             print(get_precursor(cluster_seq + 'DL', charge), obs_prec + tol)
            merges = grab_matches(cluster,ind_b, diff, 'y')
            [merged_seqs.append(x) for x in merges]

    merged_seqs.sort(key=lambda a: a[0], reverse=True)
    return merged_seqs

#All inputs are the same
merged_seqs = clustering.get_hybrid_matches(b_sorted_clusters, y_sorted_clusters, input_spectrum.precursor_mass, precursor_tolerance, input_spectrum.precursor_charge)
print(len(merged_seqs))
[print(x) for x in merged_seqs[:50]]
with open(os.path.join(location, 'merged_seqs.txt'), 'w') as b:
    [b.write(str(x) + '\n') for x in merged_seqs]

In [ ]:
def hybrid_filter_by_precursor(mseqs, obs_prec, precursor_tol, charge):
    filtered_seqs = []
    for comb_seq in mseqs:
        b_seq = comb_seq[3][4]
        y_seq = comb_seq[4][4]
        if b_seq != y_seq:
            new_seq = b_seq + y_seq
        else:
            new_seq = b_seq
        tol = utils.ppm_to_da(obs_prec, precursor_tol)
        if not (get_precursor(new_seq, charge) > obs_prec + tol):
            filtered_seqs.append(comb_seq)
    return filtered_seqs

merged_seqs = hybrid_filter_by_precursor(merged_seqs, input_spectrum.precursor_mass, precursor_tolerance, input_spectrum.precursor_charge)
print(len(merged_seqs))
[print(x) for x in merged_seqs[:50]]

In [ ]:
def get_overlapping_sequence(b_seq, y_seq, b_start, b_end, y_start):
    seq = ''
    if y_start > b_end:
        return b_seq + y_seq
    else:
        for i in range(b_start, y_start):
            seq = seq + b_seq[i]
        return seq
def overlap(comb_seq):
    b_seq = comb_seq[3][4]
    y_seq = comb_seq[4][4]
    b_pid = comb_seq[3][0]
    y_pid = comb_seq[4][0]
    if b_pid == y_pid:
        y_start = comb_seq[4][1]
        b_end = comb_seq[3][2]
        if (y_start - b_end > 0) & (y_start - b_end < 10):
            b_start = comb_seq[3][1]
            return get_overlapping_sequence(b_seq, y_seq, b_start, b_end, y_start)
        else:
            return b_seq + y_seq
    else:
        return b_seq + y_seq

def comb_seq_precursor(comb_seq, obs_prec, precursor_tol, charge): #Currently unused. Created for testing how close initial hits are to the precursor
    new_seq = overlap(comb_seq)
    tol = utils.ppm_to_da(obs_prec, precursor_tol)
    return obs_prec + tol - get_precursor(new_seq, charge)

def find_next_mass(comb_seq, ion):
    if ion == 'b':
        b_tup = comb_seq[3]
        target_index = b_tup[2]
        target_prot = b_tup[0]
        for i, prot_name in enumerate(db.proteins):
            if i == target_prot:
                protein = db.proteins[prot_name]
                prot_seq = protein[0][1]
                to_add = prot_seq[target_index] if target_index < len(prot_seq) else ''
                    
    else:
        y_tup = comb_seq[4]
        target_index = y_tup[1]
        target_prot = y_tup[0]
        for i, prot_name in enumerate(db.proteins):
            if i == target_prot:
                protein = db.proteins[prot_name]
                prot_seq = protein[0][1]
                to_add = prot_seq[target_index] if target_index < len(prot_seq) else ''
    
    return to_add

def in_tol(mass, tolerance, queried_mass):
    if queried_mass >= mass - tol:
        if queried_mass <= mass + tol:
            return True
    return False
def filter_by_missing_mass(mseqs, obs_prec, precursor_tol, charge):
    filtered_seqs = []
    for comb_seq in mseqs:
        new_seq = overlap(comb_seq)
        tol = utils.ppm_to_da(obs_prec, precursor_tol)
        dif = obs_prec + tol - get_precursor(new_seq, charge)
#         if in_tol(obs_prec, tol, get_precursor(new_seq, charge))
        if dif <= 1: #tol can vary but i'm not sure how much. Tol is .05 for spec 4 Other hacks are 2*tol
#             print(obs_prec, get_precursor(new_seq, charge), comb_seq)
            filtered_seqs.append(comb_seq)
        else:
            next_b = find_next_mass(comb_seq, 'b')
            b_seq = comb_seq[3][4]
            y_seq = comb_seq[4][4]
            b_dif = obs_prec + tol - get_precursor(b_seq + next_b + y_seq, charge)
            next_y = find_next_mass(comb_seq, 'y')
            y_dif = obs_prec + tol - get_precursor(b_seq + next_y + y_seq, charge)
            if b_dif >= 0 and y_dif >= 0:
                filtered_seqs.append(comb_seq)
                
#             if in_tol(obs_prec, tol, get_precursor(b_seq + next_b + y_seq, charge)): #For when the entire alignment is coded
#                 filtered_seqs.append(comb_seq)
#             else:

    return filtered_seqs

merged_seqs = filter_by_missing_mass(merged_seqs, input_spectrum.precursor_mass, precursor_tolerance, input_spectrum.precursor_charge) 
print(len(merged_seqs))
[print(x) for x in merged_seqs]

# Alignments

In [ ]:
for i, comb_seq in enumerate(merged_seqs):
    b_seq = comb_seq[3][4]
    y_seq = comb_seq[4][4]
    if b_seq == "DPQV" and y_seq == "DPQVAQLELGGGPGAG":
        print(i)

In [ ]:
def modified_find_next_mass(cluster, ion):
    if ion == 'b':
        target_index = cluster[2] + 1
    else:
        target_index = cluster[1]-1
    target_prot = cluster[0]
    for i, prot_name in enumerate(db.proteins):
        if i == target_prot:
            protein = db.proteins[prot_name]
            prot_seq = protein[0][1]
            to_add = prot_seq[target_index] if (target_index < len(prot_seq) and target_index > 0) else ''
    
    return to_add

def make_merge(b, y, b_seq, y_seq):
    new_b = (b[0], b[1], b[2], b[3], b_seq)
    new_y = (y[0], y[1], y[2], y[3], y_seq)
    return (b[3] + y[3], b[1] - y[2], y[2]-b[1], new_b, new_y)

def add_amino_acids(alignment_list, missing_mass, b_c, y_c, comb_seq, b_seq, y_seq, precursor_charge, prec_mass, tol):
    #This function recursively adds in amino acids    
    if abs(get_precursor(b_seq + y_seq, precursor_charge) - prec_mass) <= tol:
        alignment_list.add(make_merge(b_c, y_c, b_seq, y_seq))
        return
    
    if get_precursor(b_seq + y_seq, precursor_charge) > prec_mass + tol:
        return
    
    next_b = modified_find_next_mass(b_c, 'b')
    next_y = modified_find_next_mass(y_c, 'y')
    
    if get_precursor(b_seq + y_seq, precursor_charge) < prec_mass - tol and (next_b != ""):
        mod_b = b_seq + next_b
        mod_b_c = (b_c[0], b_c[1], b_c[2]+1, b_c[3], mod_b)
        add_amino_acids(alignment_list, missing_mass, mod_b_c, y_c, comb_seq, mod_b, y_seq, precursor_charge, prec_mass, tol)
    if get_precursor(b_seq + y_seq, precursor_charge) < prec_mass - tol and (next_y != ""):
        mod_y = next_y + y_seq
        mod_y_c = (y_c[0], y_c[1]-1, y_c[2], y_c[3], mod_y)
        add_amino_acids(alignment_list, missing_mass, b_c, mod_y_c, comb_seq, b_seq, mod_y, precursor_charge, prec_mass, tol)
        
    return

        
def find_alignments(merged_seqs, obs_prec, prec_charge, tol):
    alignments = set()
    for comb_seq in merged_seqs:
        b_cluster = comb_seq[3]
        y_cluster = comb_seq[4]
        b_seq = comb_seq[3][4]
        y_seq = comb_seq[4][4]
        if b_seq != y_seq:
            new_seq = b_seq + y_seq
            missing_mass = obs_prec - get_precursor(new_seq, prec_charge)
            add_amino_acids(alignments, missing_mass, b_cluster, y_cluster, comb_seq, b_seq, y_seq, prec_charge, obs_prec, tol)            
        else:
            new_seq = b_seq
            if (abs(get_precursor(new_seq, prec_charge) - obs_prec) <= tol):
                alignments.add(comb_seq)
            
    return alignments

tol = utils.ppm_to_da(input_spectrum.precursor_mass, precursor_tolerance)
alignments = find_alignments(merged_seqs[:50], input_spectrum.precursor_mass, input_spectrum.precursor_charge, tol)
print(alignments)

# Second Round of Scoring

In [ ]:
def score_by_dist(merged_seqs, obs_prec, precursor_tol, charge):
    new_list = []
    for comb_seq in merged_seqs:
        b_seq = comb_seq[3][4]
        y_seq = comb_seq[4][4]
        if b_seq != y_seq:
            new_seq = b_seq + y_seq
        else:
            new_seq = b_seq
        tol = utils.ppm_to_da(obs_prec, precursor_tol)
        dist = abs(get_precursor(new_seq, charge) - obs_prec)
        new_list.append((dist, comb_seq))
    return new_list
    
print("SpecMill Sequence Dist:", abs(input_spectrum.precursor_mass - get_precursor(correct_sequence, input_spectrum.precursor_charge)))
new_merges = score_by_dist(alignments, input_spectrum.precursor_mass, 10, input_spectrum.precursor_charge)
new_merges.sort(key=lambda a: a[0])
[print(x) for x in new_merges[:50]]

In [ ]:
def find_total_score(sequence, i_spectrum, ppm_tol):
    total_score = 0
    spectrum = gen_spectrum(sequence)
    masses = sorted(spectrum['spectrum'])
    o_ctr, t_ctr = 0, 0
    observed = i_spectrum[o_ctr]
    theoretical = masses[t_ctr]
    tol = utils.ppm_to_da(observed, ppm_tol)
    while (o_ctr < len(i_spectrum) and t_ctr < len(masses)):
        if theoretical < observed - tol:
            t_ctr = t_ctr + 1
            if t_ctr < len(masses):
                theoretical = masses[t_ctr]
        elif observed + tol < theoretical:
            o_ctr = o_ctr + 1
            if o_ctr < len(i_spectrum):
                observed = i_spectrum[o_ctr]
            tol = utils.ppm_to_da(observed, ppm_tol)
        elif observed - tol <= theoretical and observed + tol >= theoretical:
            total_score = total_score + 1
            o_ctr = o_ctr + 1
            t_ctr = t_ctr + 1
            if o_ctr < len(i_spectrum) and t_ctr < len(masses):
                observed = i_spectrum[o_ctr]
                theoretical = masses[t_ctr]
                tol = utils.ppm_to_da(observed, ppm_tol)
        
    return(total_score)

def find_for_all(merged_seqs, i_spectrum, ppm_tol):
    new_list = []
    for comb_seq in merged_seqs:
        b_seq = comb_seq[3][4]
        y_seq = comb_seq[4][4]
        if b_seq != y_seq:
            new_seq = b_seq + y_seq
        else:
            new_seq = b_seq
        score = find_total_score(new_seq, i_spectrum, ppm_tol)
        new_list.append((score, comb_seq))
        
    new_list.sort(key=lambda a: a[0], reverse = True)
    return new_list
    
    
print("SpecMill Sequence Total Score:", find_total_score(correct_sequence, input_spectrum.mz_values, ppm_tolerance))
tol = utils.ppm_to_da(input_spectrum.precursor_mass, 10)
print(input_spectrum.precursor_mass, get_precursor(correct_sequence, input_spectrum.precursor_charge), abs(input_spectrum.precursor_mass - get_precursor(correct_sequence, input_spectrum.precursor_charge)))
rescored_merges = find_for_all(alignments, input_spectrum.mz_values, ppm_tolerance)
[print(x) for x in rescored_merges[:50]]